In [1]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 


In [2]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2026-01-16 15:02:17.609 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/max/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 7721.67it/s]


In [3]:
len(train), len(valid)

(81, 20)

In [4]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

(torch.Size([32, 41, 3]),
 tensor([ 5, 18, 19, 16,  5, 15, 14,  0,  8, 17,  6, 16,  9,  5, 17, 11, 17, 18,
         15,  4,  6, 19,  3, 18,  4, 11, 16, 16,  3,  5, 12,  2]))

Can you make sense of the shape?
What does it mean that the shapes are sometimes (32, 27, 3), but a second time might look like (32, 30, 3)? In other words, the second (or first, if you insist on starting at 0) dimension changes. Why is that? How does the model handle this? Do you think this is already padded, or still has to be padded?


# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [5]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [6]:
model = rnn_models.BaseRNN(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    horizon=20,
)

Test the model. What is the output shape you need? Remember, we are doing classification!

In [7]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])

Test the accuracy

In [8]:
accuracy(y, yhat)

0.125

What do you think of the accuracy? What would you expect from blind guessing?

Check shape of `y` and `yhat`

In [9]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

And look at the output of yhat

In [10]:
yhat[0]

tensor([-0.0762, -0.0173, -0.0097,  0.0442,  0.0669, -0.1163, -0.1243,  0.0295,
        -0.1296,  0.0849, -0.0650, -0.1527,  0.0154, -0.0643,  0.0330, -0.0378,
         0.1057,  0.0512,  0.0695, -0.0128], grad_fn=<SelectBackward0>)

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [11]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(2.9916, grad_fn=<NllLossBackward0>)

In [12]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

using cpu


Set up the settings for the trainer and the different types of logging you want

In [37]:
settings = TrainerSettings(
    epochs=12, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 12
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [38]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [39]:
config = ModelConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)


In [48]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "modelname-here")
    mlflow.set_tag("dev", "your-name-here")
    config = ModelConfig(
        input_size=3,
        hidden_size=64,
        num_layers=4,
        output_size=20,
        dropout=0.1,
    )

    model = GRUmodel(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

2026-01-16 15:35:39.596 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20260116-153539
2026-01-16 15:35:39.598 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:02<00:00, 34.83it/s]
2026-01-16 15:35:42.559 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.5999 test 2.3063 metric ['0.1437']
100%|██████████| 81/81 [00:02<00:00, 35.32it/s]
2026-01-16 15:35:45.486 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 2.1470 test 1.9735 metric ['0.2750']
100%|██████████| 81/81 [00:02<00:00, 37.84it/s]
2026-01-16 15:35:48.253 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 1.7842 test 1.5708 metric ['0.3812']
100%|██████████| 81/81 [00:02<00:00, 35.38it/s]
2026-01-16 15:35:51.177 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 1.5181 test 1.4008 metric ['0.3828']
100%|██████████| 81/81 [00:02<00:00, 34.37it/s]
2026-01-16 15:35:54.16

🏃 View run luminous-bass-388 at: http://localhost:5000/#/experiments/102469092367217827/runs/c45b22cf2d264b409aa590bc54475bd8
🧪 View experiment at: http://localhost:5000/#/experiments/102469092367217827


Try to update the code above by changing the hyperparameters.
    
To discern between the changes, also modify the tag mlflow.set_tag("model", "new-tag-here") where you add
a new tag of your choice. This way you can keep the models apart.

In [29]:
trainer.loop() # if you want to pick up training, loop will continue from the last epoch

100%|██████████| 81/81 [00:00<00:00, 150.27it/s]
2026-01-16 15:26:06.773 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.0202 test 0.1374 metric ['0.9688']
2026-01-16 15:26:06.774 | INFO     | mltrainer.trainer:__call__:252 - best loss: 0.1178, current loss 0.1374.Counter 1/5.
2026-01-16 15:26:06.775 | INFO     | mltrainer.trainer:loop:103 - Interrupting loop due to early stopping patience.
2026-01-16 15:26:06.776 | INFO     | mltrainer.trainer:loop:108 - early_stopping_save was false, using latest model.Set to true to retrieve best model.
  0%|          | 0/3 [00:01<?, ?it/s]


In [47]:
mlflow.end_run()